In [1]:
"""
SEE: https://github.com/sudoswap/lssvm/blob/main/src/LSSVMPair.sol
Events to query for are:
  1. SwapNFTInPair()
    - emitted by LSSVMPair::swapNFTsForToken (sell order)
  2. SwapNFTOutPair()
    - emitted by LSSVMPair::swapTokenForAnyNFTs (buy order)
  3. FeeUpdate()
    - emitted by LSSVMPair::changeFee
  4. DeltaUpdate()
    - emitted by LSSVMPair::changeDelta, ::swapNFTsForToken, ::swapTokenForAnyNFTs

Pool analysis for a given pair contract.

SHOULD replicate https://github.com/sudoswap/pools-analysis but with historical charts for
pool performance.
"""
# switch to sudo-ape wd first for ape project to work properly
import os
from pathlib import Path

if Path(*Path(os.getcwd()).parts[-2:]) == Path('sudo-ape/notebook'):
    os.chdir('..')

In [3]:
import numpy as np
import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks, project

In [4]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [6]:
start_block = 14645816  # pair factory deployment
start_block

14645816

In [10]:
# TODO: change to whatever pool address you want
pool_addr = "0x5caf332dca4e6c9e69d52f320c21e74845353db0"  # xmons
pool = project.LSSVMPair.at(pool_addr)

In [11]:
pool

<LSSVMPair 0x5caF332DcA4e6C9E69d52F320c21e74845353db0>

In [13]:
# curve type
curve = Contract(pool.bondingCurve())
curve

<LinearCurve 0x5B6aC51d9B1CeDE0068a1B26533CAce807f883Ee>

In [39]:
# collection
collection = project.ERC721.at(pool.nft())
collection

<ERC721 0x0427743DF720801825a5c82e0582B1E915E0F750>

In [43]:
# utility functions to unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict) -> tp.Any:
    return event_args[key]

def unfold(df: pd.DataFrame):
    if df.empty:
        return
    for key in df.iloc[-1]['event_arguments'].keys():
        df[key] = df['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [34]:
# query for historical NFT in swap in events
%time query_swaps_in = pool.SwapNFTInPair.query('*', start_block=start_block)
query_swaps_in[:10]

CPU times: user 35.7 s, sys: 5.46 s, total: 41.2 s
Wall time: 5min 32s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x195069bfc2a5e1ee78ee3cd8114fdfce8c0d482036d3...,15198846,0xd3b29ce639c43280c2a194b638e07a5e1541824210aa...,57,37
1,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x414ea41198833a399a68c9b66456047bc6dd349330d7...,15208415,0x36dce9b484abf8db83699dabf5d63cbca4b9f68cc7e2...,123,56
2,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x28758b8d0ad21047a93b75089e210e0ca85f652842ef...,15218330,0x10112e7b56853191c7e46d36f0cca7431be567982f5c...,76,59
3,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x0b7981b1331274e198c6e5aee0b0cf568f71c3946e9b...,15232365,0xa5894445a3e40d13ffede5d8de5b0792637dff7db649...,183,126
4,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x75465069ecca43fc7daba4aefd33853c8968aeac7fc4...,15279912,0xe1d0b65d2ef045dde8abe7471b04e3f746091c611f33...,60,24
5,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0xf73cc2c43140b21129064288e080279b447ef73dcd38...,15284547,0x31740c7a4be1ff152694580b68a768ceb246bc4ec9e5...,313,84
6,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x3023d7e712763778f0f49d8723fa18998f3527e3d4b8...,15291179,0xe124608a4134e9e076de26966ee9d522991312f7253e...,104,71
7,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x8f2d12d443584412eb7b5191d2413e4098134c3b418b...,15297928,0x974ce300f99fadf0b083c33deb3253b1513ad2e85316...,157,61
8,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x446121eff4476fa89518cd9c77417f907ef3a783c67a...,15303039,0x278b90842efd2a6df0d9f63b6d50baba7f8c9c8606f0...,18,1
9,SwapNFTInPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0xd2cd286dea71b61d328b94fd08756d0519dc7e3ed802...,15321994,0xa85b565ce973c82b3ac204d24a003f539c90a2a0c451...,251,141


In [35]:
# query for historical NFT out swap events
%time query_swaps_out = pool.SwapNFTOutPair.query('*', start_block=start_block)
query_swaps_out[:10]

CPU times: user 37.3 s, sys: 5.7 s, total: 43 s
Wall time: 5min 48s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0xe6d9db380c0b7b3d9923ac18e1a920c64a60ef0e2f2e...,15199800,0x420f1997f04b486a8ed114b32337abb0f8722e82ef1f...,82,82
1,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x85212d1b00c2c505bda6b7a56d2b1f2a893646a0d55b...,15203036,0x0e89e72ff162aa1b1ec18cdad70825885d7c0884f425...,236,138
2,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x81073cf223fe0c5be5c4d6c1898033089431f6d1e57c...,15221322,0x56157a3edc0d20332ce0a89d77a1f4d79a80b2eb7847...,71,74
3,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0xed22789a0fb2b93e0cc03f87c8a3d56903c9d5175b90...,15222022,0xefe63b24d1ced117c27382616f8faef81b12560e69bf...,379,203
4,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x01b94ecd67865f46b8f70b02ecac42b33092df846b67...,15231688,0xbb581b19ccba192f1174709e78afc66145999255e78f...,117,64
5,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x400e1b05d002bff37f59d191e544a9580bcf0a74a88b...,15279524,0xd01b0d5c2a04b37c5ac064b569be4fc2d3d010a64ecd...,316,168
6,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x3051550dda227e6d301ab444dd95553dab30b75726da...,15280287,0x112c11e5fbc3b6c835f742a676f5772afb4fd622c010...,154,52
7,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x5c9d27d3ea06332c38e8a6fcae54a171fe92492404f8...,15291055,0x8ad9a6bd6c5dae22abd42fcef1604d7cb5681983c746...,284,138
8,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x6d95f002549235171dedfc3063e9cc51b76bb23a72fd...,15306841,0x60aa9716efbfb913846d871b7181be452273479d7f95...,523,515
9,SwapNFTOutPair,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{},0x33c9c5b3cb79dd803b74b57acb228bc0033635b64145...,15310405,0x3660dfc5de165ec8af62f1787072ebccf6f060dbe1b1...,130,76


In [36]:
# combine the two queries sorting by (block_number, transaction_index)
query_swaps = pd.concat([query_swaps_in, query_swaps_out]).sort_values(by=['block_number', 'transaction_index']).set_index('block_number')

In [37]:
# get the transaction hashes from each swap to examine transfer events
query_swaps['transaction_hash']

block_number
15198846    0x195069bfc2a5e1ee78ee3cd8114fdfce8c0d482036d3...
15199800    0xe6d9db380c0b7b3d9923ac18e1a920c64a60ef0e2f2e...
15203036    0x85212d1b00c2c505bda6b7a56d2b1f2a893646a0d55b...
15208415    0x414ea41198833a399a68c9b66456047bc6dd349330d7...
15218330    0x28758b8d0ad21047a93b75089e210e0ca85f652842ef...
                                  ...                        
15583837    0x0f0c92c52a66419cf127e6d95465200306c53ba0f614...
15622000    0x3343670d15a90410dd5f689d9e6349e2a4343b7472e0...
15630253    0xb11f71335fc38495ae8e9a3dac52bf8cf452f7b02c74...
15641940    0xa1759dc5b910d93a3464b9814e8010bddd25dafe66d7...
15642260    0xa29e107e4b2b8ab807c281e02b2ab8619e4bc673de26...
Name: transaction_hash, Length: 73, dtype: object

In [42]:
# query transfer events for collection
%time query_transfers = collection.Transfer.query('*', start_block=start_block)
query_transfers[:10]

CPU times: user 36 s, sys: 5.52 s, total: 41.5 s
Wall time: 5min 30s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0xF3dca43787d77f846974BE7d75BAeE8AaB...,0xffe959b85f7c31ea40c9e81c7211018fba4b65173e77...,14649722,0x404434ad5238096cbb5354fd7e4f26838256af2d3d87...,113,110
1,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x75d4bdBf6593ed463e9625694272a0FF9a...,0xff03f1a4956f878f1bd42a6dfa2ec18165822cde3d15...,14652223,0x877d4993fe635aaaea71fa3c8bb8c705d67f86559c3e...,141,108
2,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x0636a6Bd07dDBEA28a3cc63353ea725e8c...,0x2d52cdf7294e2abf00ba2c9aeb124f1119d376ad5490...,14652258,0x45bc8a840e7b5353c8967fc6152c279e1ca74df4ec02...,13,7
3,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x008f4C31734052152AddA6afDaD6955379...,0x0398cfe92a9de73c6d6adf566e5af8f69b20ddc5725b...,14652290,0xd40960785cb305acb95b58dc760157855d0811f1195f...,109,45
4,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0xbB0f753321e2B5FD29Bd1d14b532f5B549...,0xe3920493fcb9618cc94008490c030255ae9aa9fd1e45...,14671799,0x0d590ad40f1cc782d2703f6e0b62cd6acaa13dceeb98...,231,91
5,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0xd48D8cef2F1A7b29BAFb5E17e8B88bfEBa...,0xbeb2daa3907185013eceb28a72dffc87815a057d209a...,14694998,0x38120ee9a8a1b57e09461467ec6ea7903b6a12a400be...,373,297
6,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x83C8F28c26bF6aaca652Df1DbBE0e1b56F...,0xbeb2daa3907185013eceb28a72dffc87815a057d209a...,14694998,0x38120ee9a8a1b57e09461467ec6ea7903b6a12a400be...,376,297
7,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0xCd723Fa1BF4D1258d5d6C9d5149e85acfD...,0x37de4c4c8c523ec6633f5d1293c96ed142894a65c6fc...,14713238,0xc80600afee6d241acc39300c502b30d294a12db5eeef...,295,83
8,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x80c939F8A66C59B37330f93f1002541fD4...,0x45767aa315dc2f1f9c215fb1fee239136e7e23f14795...,14747031,0x556aa6640702bd8a0b8a40d54138d5e3aadc9f3970ea...,199,86
9,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x6327343B3a12Fbd25488Da0e82Bd4512aD...,0x1cad6649f4c3c17011cbd033455d7b7ea0d66786e503...,14748679,0xc20d89ae4f3e801415b6417590c7a6f9c996e585fdbc...,195,128


In [45]:
# unfold event args
unfold(query_transfers)

In [54]:
# filter collection transfer events based on swap tx hashes
query_transfers_for_swaps = query_transfers[query_transfers['transaction_hash'].isin(query_swaps['transaction_hash'])].sort_values(by=['block_number', 'transaction_index', 'log_index']).set_index('block_number')
query_transfers_for_swaps

,event_name,contract_address,event_arguments,transaction_hash,block_hash,log_index,transaction_index,from,to,id
block_number,,,,,,,,,,
15198846,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x78D0bAC4E33A047224c5056Cb9891D2816...,0x195069bfc2a5e1ee78ee3cd8114fdfce8c0d482036d3...,0xd3b29ce639c43280c2a194b638e07a5e1541824210aa...,56,37,0x78D0bAC4E33A047224c5056Cb9891D28168F67c4,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,66
15199800,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x5caF332DcA4e6C9E69d52F320c21e74845...,0xe6d9db380c0b7b3d9923ac18e1a920c64a60ef0e2f2e...,0x420f1997f04b486a8ed114b32337abb0f8722e82ef1f...,81,82,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,0xda3845B44736b57E05Ee80fC011A52a9c777423A,267
15203036,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x5caF332DcA4e6C9E69d52F320c21e74845...,0x85212d1b00c2c505bda6b7a56d2b1f2a893646a0d55b...,0x0e89e72ff162aa1b1ec18cdad70825885d7c0884f425...,235,138,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,0x85B44a83bCa937E844Bdc290284Ef6b11BC015ba,128
15208415,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x883F4BF1Cd53692Bb602BFb580d4B15045...,0x414ea41198833a399a68c9b66456047bc6dd349330d7...,0x36dce9b484abf8db83699dabf5d63cbca4b9f68cc7e2...,122,56,0x883F4BF1Cd53692Bb602BFb580d4B1504539dFb8,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,323
15218330,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x7A69a8399740fcB54a182957181a5FCA67...,0x28758b8d0ad21047a93b75089e210e0ca85f652842ef...,0x10112e7b56853191c7e46d36f0cca7431be567982f5c...,75,59,0x7A69a8399740fcB54a182957181a5FCA675e1eF5,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,22
...,...,...,...,...,...,...,...,...,...,...
15583837,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x6291bD1Ecb88D5C8f6702D6Faa91661e2C...,0x0f0c92c52a66419cf127e6d95465200306c53ba0f614...,0xac80c3a0f4b64fbcb0512cf4bcbfc67f77cb37a6bd36...,84,37,0x6291bD1Ecb88D5C8f6702D6Faa91661e2C0a9e7E,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,316
15622000,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x5caF332DcA4e6C9E69d52F320c21e74845...,0x3343670d15a90410dd5f689d9e6349e2a4343b7472e0...,0x76e15f58bc7e39a5e164277fa0b2bc2c6648f510adf7...,73,47,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,0x14df40586e0F2Ca5CA222DB0dAc8Df2f4C5b1AAb,189
15630253,Transfer,0x0427743DF720801825a5c82e0582B1E915E0F750,{'from': '0x15D3FfAc9B7D5dBbEE095910Db63800402...,0xb11f71335fc38495ae8e9a3dac52bf8cf452f7b02c74...,0x9f61426c708fcd4622f26fd23b607c65a2cf08d3c4a2...,19,26,0x15D3FfAc9B7D5dBbEE095910Db6380040222b99f,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,110
